In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
import json
import glob
import keras
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers import Dropout
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Using TensorFlow backend.


In [2]:
list_of_file_names = glob.glob("./zip_2/*.csv")

In [3]:
read_data_list_2 = []
fail_list = []
for i, data in enumerate(list_of_file_names):
    try:
        data = pd.read_csv(data)
        #data = unicode(data, errors='replace')
        read_data_list_2.append(data)
    except UnicodeDecodeError:
        fail_list.append(i)
#read_data_list
#fail_list
list_of_file_names_org = list_of_file_names

In [4]:
fail_list

[3]

In [5]:
for x in fail_list:
    del list_of_file_names[x]

In [6]:
list_of_file_names[3]

'./zip_2/Zip_Listings_PriceCut_SeasAdj_AllHomes.csv'

In [7]:
len(read_data_list_2[3])

10172

In [8]:
list_of_file_names[4]

'./zip_2/Zip_Listings_PriceCut_SeasAdj_Condominum.csv'

In [9]:
len(read_data_list_2[4])

844

In [10]:
# build loop for data we can work with. Only dates and zip
def build_useful_df(data_list):
    useful_df = []
    for dataset in range(len(data_list)):
        data = read_data_list_2[dataset]
        region_name = data['RegionName']
        new_df = data.select_dtypes(include=['float64'])
        new_df.insert(loc=0, column='RegionName', value=region_name) 
        new_df = new_df.sort_values('RegionName',ascending=False)
        new_df = new_df.set_index('RegionName')
        useful_df.append(new_df)
    return useful_df

In [11]:
useful_df = build_useful_df(read_data_list_2)
useful_df[3].head()

,2010-09,2010-10,2010-11,2010-12,2011-01,2011-02,2011-03,2011-04,2011-05,2011-06,...,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12
RegionName,,,,,,,,,,,,,,,,,,,,,
99801,5.001480,12.059667,9.721639,12.823805,12.067908,6.735467,12.817048,5.351705,7.022677,11.403385,...,8.149935,7.839226,8.340472,11.016614,13.717871,12.825190,12.298428,13.377564,11.830469,10.323291
99709,13.332107,11.447750,20.230705,17.569812,9.501848,16.866147,25.336712,14.339757,14.388552,11.076664,...,15.020218,15.179652,9.608745,15.497014,13.444893,21.787553,14.004356,17.560603,14.040229,12.349473
99705,14.238275,11.455669,21.931982,19.715816,13.152595,16.048516,16.539511,13.047547,15.500396,18.073011,...,17.252181,14.419267,13.725248,14.145342,15.184207,21.538185,15.127410,18.841846,19.314115,18.508919
99701,12.502596,3.868248,11.226388,8.029469,11.557336,13.020347,6.255375,7.774582,10.831500,12.234200,...,11.990818,11.121010,13.573817,14.626545,17.361696,14.490435,12.286012,12.098577,12.474354,16.429898
99669,12.342694,9.547881,9.399721,11.789248,13.575353,8.824402,11.448679,12.713187,12.052649,9.105020,...,8.814888,10.137303,13.932264,12.073197,9.835698,14.488406,11.190449,15.390050,11.528148,13.687983


In [22]:
# concat all dfs into one multi-index. I am not currently using this
def df_build_dataframe(df_list, beginning, end):
    length = len(df_list[beginning:end])
    keys = []
    values = []
    final = []
    for x in range(length):
        y = str(x)
        keys.append(y)
    for df in df_list[beginning:end]:
        values.append(df)
    dictionary = dict(zip(keys, values))
    df = pd.concat(dictionary)
    return df
single_df = df_build_dataframe(useful_df, 3, 71)

In [23]:
# Building a list of all dataframes that work seamlessly with function. Tested through trial and error to get these 
# values
good_range = list(range(72))
good_range.remove(0)
good_range.remove(1)
good_range.remove(2)

In [12]:
# Building the features for the model. The ones commented out I want to work on after I run the initial model
# They will have more parameters for instance, "did this zipcode have a big swing and gain value?"
# I also want to add some of those rolling type of features like "mean, std, min, max, swing, change, for last three months"
# Also listing any files that don't fit the date parameters in 'bugs'
def build_feature_list(good_range, df_list):
    year_list = ['2011', '2012', '2013', '2014', '2015', '2016']
    feature_df_list = []
    bugs = []
    for num in good_range:
        df = df_list[num]
        try:
            df = df.loc[:, '2011-01':'2016-12']
            features = pd.DataFrame()
            for i, year in enumerate(year_list):
                mean = df.loc[:, year + '-01': year + '-12'].mean(axis=1)
                features[year + '_mean'] = mean
                std = df.loc[:, year + '-01': year + '-12'].std(axis=1)
                features[year + '_std'] = std
                mn = df.loc[:, year + '-01': year + '-12'].min(axis=1)
                features[year + '_min'] = mn
                mx = df.loc[:, year + '-01': year + '-12'].max(axis=1)
                features[year + '_max'] = mx
                features[year + '_swing'] = mx - mn
                change = df[year + '-12'] - df[year + '-01']
                features[year + '_change'] = change
                if i > 0:
                    yoy = features[year + '_mean'] / df.loc[:, year_list[i - 1] + '-01': year_list[i - 1] + '-12'].mean(axis=1)
                    features[year + '_yoy'] = yoy
                    features[year + '_gain'] = np.where(features[year + '_yoy']>1, 1, 0)
                    
                    #big swing and gain, big swing and loss, big swing and big gain, big swing and big loss
            feature_df_list.append(features)
        except:
            bugs.append(num)
    return feature_df_list, bugs

In [13]:
# Building the features for the model. The ones commented out I want to work on after I run the initial model
# They will have more parameters for instance, "did this zipcode have a big swing and gain value?"
# I also want to add some of those rolling type of features like "mean, std, min, max, swing, change, for last three months"
# Also listing any files that don't fit the date parameters in 'bugs'
def build_feature_list_swing(good_range, df_list):
    year_list = ['2011', '2012', '2013', '2014', '2015', '2016']
    feature_df_list = []
    bugs = []
    for num in good_range:
        df = df_list[num]
        try:
            df = df.loc[:, '2011-01':'2016-12']
            features = pd.DataFrame()
            for i, year in enumerate(year_list):
                mean = df.loc[:, year + '-01': year + '-12'].mean(axis=1)
                features[year + '_mean'] = mean
                std = df.loc[:, year + '-01': year + '-12'].std(axis=1)
                features[year + '_std'] = std
                mn = df.loc[:, year + '-01': year + '-12'].min(axis=1)
                features[year + '_min'] = mn
                mx = df.loc[:, year + '-01': year + '-12'].max(axis=1)
                features[year + '_max'] = mx
                features[year + '_swing'] = mx - mn
                change = df[year + '-12'] - df[year + '-01']
                features[year + '_change'] = change
                if i > 0:
                    yoy = features[year + '_mean'] / df.loc[:, year_list[i - 1] + '-01': year_list[i - 1] + '-12'].mean(axis=1)
                    features[year + '_yoy'] = yoy
                    features[year + '_gain'] = np.where(features[year + '_yoy']>1, 1, 0)
                    mean_yoy = features[year + '_yoy'].mean()
                    features[year + '_yoy_pos'] = np.where(features[year + '_yoy']>mean_yoy, 1, 0)
                    big_yoy = features[year + '_yoy'].std() + mean_yoy
                    features[year + '_yoy_big'] = np.where(features[year + '_yoy']>big_yoy, 1, 0)
                    features[year + '_yoy_neg'] = np.where(features[year + '_yoy']<mean_yoy, 1, 0)
                    big_loss = mean_yoy - features[year + '_yoy'].std() 
                    features[year + '_yoy_loss_big'] = np.where(features[year + '_yoy']<big_loss, 1, 0)
                mean_swing = features[year + '_swing'].mean()
                features[year + '_swing_pos'] = np.where(features[year + '_swing']>mean_swing, 1, 0)
                big_swing = features[year + '_swing'].std() + mean_swing
                features[year + '_swing_big'] = np.where(features[year + '_swing']>big_swing, 1, 0)
                features[year + '_swing_neg'] = np.where(features[year + '_swing']<mean_swing, 1, 0)
                swing_big_loss = mean_swing - features[year + '_swing'].std() 
                features[year + '_swing_loss_big'] = np.where(features[year + '_swing']<swing_big_loss, 1, 0)
                    #big swing and gain, big swing and loss, big swing and big gain, big swing and big loss
            feature_df_list.append(features)
        except:
            bugs.append(num)
    return feature_df_list, bugs

In [14]:
feature_df_list, bug_list = build_feature_list([23], useful_df)

In [15]:
feature_df_list_swing, bug_list_swing = build_feature_list_swing([23], useful_df)

In [496]:
feature_df_list_mult, bug_list_mult = build_feature_list([3, 6, 14, 23, 27, 35, 46], useful_df)

In [497]:
feature_df_list_swing_mult, bug_list_swing_mult = build_feature_list_swing([3, 6, 14, 23, 27, 35, 46], useful_df)

In [458]:
feature_df_list_swing[0].head()

,2011_mean,2011_std,2011_min,2011_max,2011_swing,2011_change,2011_swing_pos,2011_swing_big,2011_swing_neg,2011_swing_loss_big,...,2016_yoy,2016_gain,2016_yoy_pos,2016_yoy_big,2016_yoy_neg,2016_yoy_loss_big,2016_swing_pos,2016_swing_big,2016_swing_neg,2016_swing_loss_big
RegionName,,,,,,,,,,,,,,,,,,,,,
99901,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,...,1.090750,1,1,0,0,0,0,0,1,0
99801,188.225207,7.093943,172.811060,197.583313,24.772253,3.408637,1,0,0,0,...,0.999862,0,0,0,1,0,0,0,1,0
99709,184.281590,14.153608,169.023628,208.530806,39.507178,-39.507178,1,1,0,0,...,1.067973,1,1,0,0,0,1,0,0,0
99705,164.434369,3.401094,161.103048,172.209738,11.106690,-1.720695,0,0,1,0,...,1.061596,1,1,0,0,0,0,0,1,0
99701,124.640998,5.854324,110.247445,131.628788,21.381343,-8.482437,1,0,0,0,...,1.166714,1,1,1,0,0,1,0,0,0


In [16]:
len(feature_df_list)
for num in range(len(feature_df_list)):
    feature_df_list[num].dropna(inplace=True)
#feature_df_list.dropna(inplace=True)

In [17]:
len(feature_df_list_swing)
for num in range(len(feature_df_list_swing)):
    feature_df_list_swing[num].dropna(inplace=True)
#feature_df_list.dropna(inplace=True)

In [500]:
len(feature_df_list_mult)
for num in range(len(feature_df_list_mult)):
    feature_df_list_mult[num].dropna(inplace=True)
#feature_df_list.dropna(inplace=True)

In [501]:
len(feature_df_list_swing_mult)
for num in range(len(feature_df_list_swing_mult)):
    feature_df_list_swing_mult[num].dropna(inplace=True)
#feature_df_list.dropna(inplace=True)

In [502]:
len(feature_df_list[0])

6670

In [31]:
bug_list

[34, 45, 48, 52, 59, 60, 61, 62, 63, 64, 65, 67, 68, 70, 71]

In [162]:
# Merging all dfs into one. Merging on indez which is zipcode. I was hoping for inner join but it looks like
# There are many zipcodes that only exist in certain dfs. Im hoping that reducing them to metro areas will fix this
def merge_dataframes(feature_df_list):
    df_1 = feature_df_list[0]
    for df in feature_df_list[1:]:
        df_1 = pd.merge(df_1, df, left_index=True, right_index=True, how='outer')
    
    return df_1

In [163]:
X = merge_dataframes(feature_df_list)

In [503]:
df_one_sm = pd.merge(feature_df_list_swing_mult[0], feature_df_list_swing_mult[1], left_index=True, right_index=True, how='inner')
df_two_sm = pd.merge(df_one_sm, feature_df_list_swing_mult[2], left_index=True, right_index=True, how='inner')
df_three_sm = pd.merge(df_two_sm, feature_df_list_swing_mult[3], left_index=True, right_index=True, how='inner')
df_four_sm = pd.merge(df_three_sm, feature_df_list_swing_mult[4], left_index=True, right_index=True, how='inner')
df_five_sm = pd.merge(df_four_sm, feature_df_list_swing_mult[5], left_index=True, right_index=True, how='inner')
#df_six = pd.merge(df_five, feature_df_list[6], left_index=True, right_index=True, how='inner')

In [504]:
df_one_m = pd.merge(feature_df_list_mult[0], feature_df_list_mult[1], left_index=True, right_index=True, how='inner')
df_two_m = pd.merge(df_one_m, feature_df_list_mult[2], left_index=True, right_index=True, how='inner')
df_three_m = pd.merge(df_two_m, feature_df_list_mult[3], left_index=True, right_index=True, how='inner')
df_four_m = pd.merge(df_three_m, feature_df_list_mult[4], left_index=True, right_index=True, how='inner')
df_five_m = pd.merge(df_four_m, feature_df_list_mult[5], left_index=True, right_index=True, how='inner')
#df_six = pd.merge(df_five, feature_df_list[6], left_index=True, right_index=True, how='inner')

In [505]:
len(df_four_sm)

2733

In [151]:
list_of_file_names[24:50]
# 3, 6, 14, 23, 27, 35, 46, 

['./zip_2/Zip_MedianListingPricePerSqft_CondoCoop.csv',
 './zip_2/Zip_MedianListingPricePerSqft_DuplexTriplex.csv',
 './zip_2/Zip_MedianListingPricePerSqft_Sfr.csv',
 './zip_2/Zip_MedianPctOfPriceReduction_AllHomes.csv',
 './zip_2/Zip_MedianPctOfPriceReduction_Condominum.csv',
 './zip_2/Zip_MedianPctOfPriceReduction_SingleFamilyResidence.csv',
 './zip_2/Zip_MedianRentalPrice_1Bedroom.csv',
 './zip_2/Zip_MedianRentalPrice_2Bedroom.csv',
 './zip_2/Zip_MedianRentalPrice_3Bedroom.csv',
 './zip_2/Zip_MedianRentalPrice_4Bedroom.csv',
 './zip_2/Zip_MedianRentalPrice_5BedroomOrMore.csv',
 './zip_2/Zip_MedianRentalPrice_AllHomes.csv',
 './zip_2/Zip_MedianRentalPrice_CondoCoop.csv',
 './zip_2/Zip_MedianRentalPrice_DuplexTriplex.csv',
 './zip_2/Zip_MedianRentalPrice_Mfr5Plus.csv',
 './zip_2/Zip_MedianRentalPrice_Sfr.csv',
 './zip_2/Zip_MedianRentalPrice_Studio.csv',
 './zip_2/Zip_MedianRentalPricePerSqft_1Bedroom.csv',
 './zip_2/Zip_MedianRentalPricePerSqft_2Bedroom.csv',
 './zip_2/Zip_MedianRent

In [31]:
def create_y_ratio(df, col_1, col_2):
    y = pd.DataFrame()
    y['RegionName'] = df['RegionName']
    y['y'] = df[col_2] / df[col_1]
    y = y.sort_values('RegionName',ascending=False)
    y = y.set_index('RegionName')
    return y

In [22]:
y.describe()

,y
count,10454.000000
mean,1.059853
std,0.081299
min,0.635848
25%,1.013011
50%,1.056295
75%,1.104116
max,1.438448


In [32]:
y = create_y_ratio(read_data_list_2[23], '2016-12', '2017-12')

In [50]:
def create_y_cless(y):
    y = y
    y['y_classes'] = np.where(((y['y'] >= 1) & (y['y'] < 1.1), 4), 
         (np.where((y['y'] >= 1.1) & (y['y'] < 1.2), 5)), 
         (np.where((y['y'] >= 1.2) & (y['y'] < 1.3), 6)),
         (np.where(y['y'] >= 1.3, 7)),
         (np.where((y['y'] < 1) & (y['y'] >= .9), 3)),
         (np.where((y['y'] < .9) & (y['y'] >= .8), 2, 1)))
    return y

In [51]:
y_test = create_y_class(y)
y_test.head

NameError: name 'create_y_class' is not defined

In [508]:
X = feature_df_list[0]

In [509]:
X_s = feature_df_list_swing[0]

In [510]:
X_m = df_four_m

In [511]:
X_sm = df_four_sm

In [512]:
X_df = pd.merge(X, y, right_index=True, left_index=True, how='inner')

In [513]:
Xm_df = pd.merge(X_m, y, right_index=True, left_index=True, how='inner')

In [514]:
Xs_df = pd.merge(X_s, y, right_index=True, left_index=True, how='inner')

In [515]:
Xsm_df = pd.merge(X_sm, y, right_index=True, left_index=True, how='inner')

In [516]:
len(second_df)

2733

In [517]:
X = X_df.iloc[:, :-1].values
y = X_df.iloc[:, -1].values

In [518]:
X_s = Xs_df.iloc[:, :-1].values
y_s = Xs_df.iloc[:, -1].values

In [519]:
X_m = Xm_df.iloc[:, :-1].values
y_m = Xm_df.iloc[:, -1].values

In [520]:
X_sm = Xsm_df.iloc[:, :-1].values
y_sm = Xsm_df.iloc[:, -1].values

In [521]:
X_df['y'].describe()

count    6670.000000
mean        1.055997
std         0.071161
min         0.635848
25%         1.013351
50%         1.051514
75%         1.094848
max         1.435638
Name: y, dtype: float64

In [522]:
Xm_df['y'].describe()

count    2733.000000
mean        1.052175
std         0.059670
min         0.747138
25%         1.014852
50%         1.048428
75%         1.088559
max         1.362434
Name: y, dtype: float64

In [523]:
X_sm.max()

3299000.0

In [524]:
y_s.max()

1.4356376395310819

In [525]:
X = X / 3299000

In [526]:
X_s = X_s / 3299000

In [527]:
X_m = X_m / 3299000

In [528]:
X_sm = X_sm / 3299000

In [529]:
y_s.shape

(6670,)

In [530]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= .2)

In [531]:
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X_s, y_s, test_size= .2)

In [532]:
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(X_m, y_m, test_size= .2)

In [533]:
X_train_sm, X_test_sm, y_train_sm, y_test_sm = train_test_split(X_sm, y_sm, test_size= .2)

In [534]:
len(y_train)

5336

In [555]:
model = Sequential()
model.add(Dense(64, input_shape=(46,), activation='relu'))
model.add(Dropout(.25))
model.add(Dense(64, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(128, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(256, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(2000, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(1, activation='linear'))
sgd = keras.optimizers.SGD(lr=0.002)
model.compile(optimizer=sgd,
              loss='mean_squared_error')

In [556]:
model.fit(x=X_train, y=y_train, 
          batch_size=2500, 
          epochs=2000, 
          verbose=1, 
          validation_data=(X_test, y_test),
          callbacks=None)

Train on 5336 samples, validate on 1334 samples
Epoch 1/2000
5336/5336 [==============================] - 5s 926us/step - loss: 1.1111 - val_loss: 1.0666
Epoch 2/2000
5336/5336 [==============================] - 3s 513us/step - loss: 1.0558 - val_loss: 1.0107
Epoch 3/2000
5336/5336 [==============================] - 3s 503us/step - loss: 1.0013 - val_loss: 0.9576
Epoch 4/2000
5336/5336 [==============================] - 3s 496us/step - loss: 0.9495 - val_loss: 0.9072
Epoch 5/2000
5336/5336 [==============================] - 3s 500us/step - loss: 0.9002 - val_loss: 0.8591
Epoch 6/2000
5336/5336 [==============================] - 3s 511us/step - loss: 0.8532 - val_loss: 0.8134
Epoch 7/2000
5336/5336 [==============================] - 3s 503us/step - loss: 0.8086 - val_loss: 0.7701
Epoch 8/2000
5336/5336 [==============================] - 3s 502us/step - loss: 0.7661 - val_loss: 0.7289
Epoch 9/2000
5336/5336 [==============================] - 3s 503us/step - loss: 0.7258 - val_loss: 0.689

5336/5336 [==============================] - 3s 489us/step - loss: 0.0056 - val_loss: 0.0050
Epoch 154/2000
5336/5336 [==============================] - 3s 486us/step - loss: 0.0056 - val_loss: 0.0050
Epoch 155/2000
5336/5336 [==============================] - 3s 486us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 156/2000
5336/5336 [==============================] - 3s 483us/step - loss: 0.0056 - val_loss: 0.0050
Epoch 157/2000
5336/5336 [==============================] - 3s 494us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 158/2000
5336/5336 [==============================] - 3s 482us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 159/2000
5336/5336 [==============================] - 3s 486us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 160/2000
5336/5336 [==============================] - 3s 483us/step - loss: 0.0056 - val_loss: 0.0050
Epoch 161/2000
5336/5336 [==============================] - 3s 485us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 162/2000
5336/5336 [=================

Epoch 229/2000
5336/5336 [==============================] - 3s 482us/step - loss: 0.0055 - val_loss: 0.0051
Epoch 230/2000
5336/5336 [==============================] - 3s 485us/step - loss: 0.0055 - val_loss: 0.0051
Epoch 231/2000
5336/5336 [==============================] - 3s 496us/step - loss: 0.0055 - val_loss: 0.0051
Epoch 232/2000
5336/5336 [==============================] - 3s 486us/step - loss: 0.0055 - val_loss: 0.0051
Epoch 233/2000
5336/5336 [==============================] - 3s 483us/step - loss: 0.0056 - val_loss: 0.0051
Epoch 234/2000
5336/5336 [==============================] - 3s 492us/step - loss: 0.0055 - val_loss: 0.0051
Epoch 235/2000
5336/5336 [==============================] - 3s 483us/step - loss: 0.0055 - val_loss: 0.0051
Epoch 236/2000
5336/5336 [==============================] - 3s 485us/step - loss: 0.0055 - val_loss: 0.0051
Epoch 237/2000
5336/5336 [==============================] - 3s 483us/step - loss: 0.0056 - val_loss: 0.0051
Epoch 238/2000
5336/5336 [==

Epoch 305/2000
5336/5336 [==============================] - 3s 482us/step - loss: 0.0055 - val_loss: 0.0051
Epoch 306/2000
5336/5336 [==============================] - 3s 485us/step - loss: 0.0055 - val_loss: 0.0051
Epoch 307/2000
5336/5336 [==============================] - 3s 483us/step - loss: 0.0055 - val_loss: 0.0051
Epoch 308/2000
5336/5336 [==============================] - 3s 496us/step - loss: 0.0055 - val_loss: 0.0051
Epoch 309/2000
5336/5336 [==============================] - 3s 480us/step - loss: 0.0055 - val_loss: 0.0051
Epoch 310/2000
5336/5336 [==============================] - 3s 482us/step - loss: 0.0055 - val_loss: 0.0051
Epoch 311/2000
5336/5336 [==============================] - 3s 482us/step - loss: 0.0055 - val_loss: 0.0051
Epoch 312/2000
5336/5336 [==============================] - 3s 482us/step - loss: 0.0056 - val_loss: 0.0051
Epoch 313/2000
5336/5336 [==============================] - 3s 480us/step - loss: 0.0055 - val_loss: 0.0051
Epoch 314/2000
5336/5336 [==

Epoch 381/2000
5336/5336 [==============================] - 3s 485us/step - loss: 0.0055 - val_loss: 0.0051
Epoch 382/2000
5336/5336 [==============================] - 3s 484us/step - loss: 0.0055 - val_loss: 0.0051
Epoch 383/2000
5336/5336 [==============================] - 3s 485us/step - loss: 0.0055 - val_loss: 0.0051
Epoch 384/2000
5336/5336 [==============================] - 3s 482us/step - loss: 0.0055 - val_loss: 0.0051
Epoch 385/2000
5336/5336 [==============================] - 3s 482us/step - loss: 0.0055 - val_loss: 0.0051
Epoch 386/2000
5336/5336 [==============================] - 3s 491us/step - loss: 0.0055 - val_loss: 0.0051
Epoch 387/2000
5336/5336 [==============================] - 3s 486us/step - loss: 0.0054 - val_loss: 0.0051
Epoch 388/2000
5336/5336 [==============================] - 3s 482us/step - loss: 0.0056 - val_loss: 0.0051
Epoch 389/2000
5336/5336 [==============================] - 3s 485us/step - loss: 0.0055 - val_loss: 0.0051
Epoch 390/2000
5336/5336 [==

...

5336/5336 [==============================] - 3s 526us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 1290/2000
5336/5336 [==============================] - 3s 484us/step - loss: 0.0054 - val_loss: 0.0050
Epoch 1291/2000
5336/5336 [==============================] - 3s 482us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 1292/2000
5336/5336 [==============================] - 3s 484us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 1293/2000
5336/5336 [==============================] - 3s 480us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 1294/2000
5336/5336 [==============================] - 3s 482us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 1295/2000
5336/5336 [==============================] - 3s 484us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 1296/2000
5336/5336 [==============================] - 3s 492us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 1297/2000
5336/5336 [==============================] - 3s 483us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 1298/2000
5336/5336 [========

5336/5336 [==============================] - 3s 484us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 1440/2000
5336/5336 [==============================] - 3s 498us/step - loss: 0.0054 - val_loss: 0.0050
Epoch 1441/2000
5336/5336 [==============================] - 3s 496us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 1442/2000
5336/5336 [==============================] - 3s 485us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 1443/2000
5336/5336 [==============================] - 3s 483us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 1444/2000
5336/5336 [==============================] - 3s 484us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 1445/2000
5336/5336 [==============================] - 3s 495us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 1446/2000
5336/5336 [==============================] - 3s 489us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 1447/2000
5336/5336 [==============================] - 3s 485us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 1448/2000
5336/5336 [========

5336/5336 [==============================] - 3s 485us/step - loss: 0.0056 - val_loss: 0.0050
Epoch 1590/2000
5336/5336 [==============================] - 3s 481us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 1591/2000
5336/5336 [==============================] - 3s 493us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 1592/2000
5336/5336 [==============================] - 3s 486us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 1593/2000
5336/5336 [==============================] - 3s 485us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 1594/2000
5336/5336 [==============================] - 3s 489us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 1595/2000
5336/5336 [==============================] - 3s 484us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 1596/2000
5336/5336 [==============================] - 3s 485us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 1597/2000
5336/5336 [==============================] - 3s 485us/step - loss: 0.0056 - val_loss: 0.0050
Epoch 1598/2000
5336/5336 [========

5336/5336 [==============================] - 3s 495us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 1740/2000
5336/5336 [==============================] - 3s 487us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 1741/2000
5336/5336 [==============================] - 3s 483us/step - loss: 0.0054 - val_loss: 0.0050
Epoch 1742/2000
5336/5336 [==============================] - 3s 496us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 1743/2000
5336/5336 [==============================] - 3s 487us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 1744/2000
5336/5336 [==============================] - 3s 486us/step - loss: 0.0054 - val_loss: 0.0050
Epoch 1745/2000
5336/5336 [==============================] - 3s 486us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 1746/2000
5336/5336 [==============================] - 3s 483us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 1747/2000
5336/5336 [==============================] - 3s 483us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 1748/2000
5336/5336 [========

5336/5336 [==============================] - 3s 485us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 1890/2000
5336/5336 [==============================] - 3s 486us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 1891/2000
5336/5336 [==============================] - 3s 482us/step - loss: 0.0056 - val_loss: 0.0050
Epoch 1892/2000
5336/5336 [==============================] - 3s 491us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 1893/2000
5336/5336 [==============================] - 3s 482us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 1894/2000
5336/5336 [==============================] - 3s 488us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 1895/2000
5336/5336 [==============================] - 3s 483us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 1896/2000
5336/5336 [==============================] - 3s 496us/step - loss: 0.0054 - val_loss: 0.0050
Epoch 1897/2000
5336/5336 [==============================] - 3s 482us/step - loss: 0.0055 - val_loss: 0.0050
Epoch 1898/2000
5336/5336 [========

In [557]:
model = Sequential()
model.add(Dense(64, input_shape=(90,), activation='relu'))
model.add(Dropout(.25))
model.add(Dense(64, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(128, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(256, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(2000, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(1, activation='linear'))
sgd = keras.optimizers.SGD(lr=0.01)
model.compile(optimizer=sgd,
              loss='mean_squared_error')

In [558]:
model.fit(x=X_train_s, y=y_train_s, 
          batch_size=2500, 
          epochs=2000, 
          verbose=1, 
          validation_data=(X_test_s, y_test_s),
          callbacks=None)

Train on 5336 samples, validate on 1334 samples
Epoch 1/2000
5336/5336 [==============================] - 5s 906us/step - loss: 1.0742 - val_loss: 0.8664
Epoch 2/2000
5336/5336 [==============================] - 3s 483us/step - loss: 0.8218 - val_loss: 0.6511
Epoch 3/2000
5336/5336 [==============================] - 3s 483us/step - loss: 0.6200 - val_loss: 0.4853
Epoch 4/2000
5336/5336 [==============================] - 3s 485us/step - loss: 0.4641 - val_loss: 0.3586
Epoch 5/2000
5336/5336 [==============================] - 3s 482us/step - loss: 0.3450 - val_loss: 0.2620
Epoch 6/2000
5336/5336 [==============================] - 3s 485us/step - loss: 0.2540 - val_loss: 0.1897
Epoch 7/2000
5336/5336 [==============================] - 3s 482us/step - loss: 0.1856 - val_loss: 0.1357
Epoch 8/2000
5336/5336 [==============================] - 3s 491us/step - loss: 0.1340 - val_loss: 0.0961
Epoch 9/2000
5336/5336 [==============================] - 3s 488us/step - loss: 0.0967 - val_loss: 0.067

5336/5336 [==============================] - 3s 484us/step - loss: 0.0056 - val_loss: 0.0049
Epoch 154/2000
5336/5336 [==============================] - 3s 481us/step - loss: 0.0055 - val_loss: 0.0049
Epoch 155/2000
5336/5336 [==============================] - 3s 481us/step - loss: 0.0056 - val_loss: 0.0049
Epoch 156/2000
5336/5336 [==============================] - 3s 483us/step - loss: 0.0056 - val_loss: 0.0049
Epoch 157/2000
5336/5336 [==============================] - 3s 482us/step - loss: 0.0056 - val_loss: 0.0049
Epoch 158/2000
5336/5336 [==============================] - 3s 497us/step - loss: 0.0056 - val_loss: 0.0049
Epoch 159/2000
5336/5336 [==============================] - 3s 499us/step - loss: 0.0055 - val_loss: 0.0049
Epoch 160/2000
5336/5336 [==============================] - 3s 485us/step - loss: 0.0056 - val_loss: 0.0049
Epoch 161/2000
5336/5336 [==============================] - 3s 484us/step - loss: 0.0056 - val_loss: 0.0049
Epoch 162/2000
5336/5336 [=================

Epoch 229/2000
5336/5336 [==============================] - 3s 490us/step - loss: 0.0056 - val_loss: 0.0048
Epoch 230/2000
5336/5336 [==============================] - 3s 489us/step - loss: 0.0056 - val_loss: 0.0048
Epoch 231/2000
5336/5336 [==============================] - 3s 482us/step - loss: 0.0055 - val_loss: 0.0049
Epoch 232/2000
5336/5336 [==============================] - 3s 481us/step - loss: 0.0055 - val_loss: 0.0048
Epoch 233/2000
5336/5336 [==============================] - 3s 499us/step - loss: 0.0056 - val_loss: 0.0048
Epoch 234/2000
5336/5336 [==============================] - 3s 483us/step - loss: 0.0055 - val_loss: 0.0048
Epoch 235/2000
5336/5336 [==============================] - 3s 484us/step - loss: 0.0055 - val_loss: 0.0048
Epoch 236/2000
5336/5336 [==============================] - 3s 484us/step - loss: 0.0056 - val_loss: 0.0048
Epoch 237/2000
5336/5336 [==============================] - 3s 485us/step - loss: 0.0055 - val_loss: 0.0048
Epoch 238/2000
5336/5336 [==

Epoch 305/2000
5336/5336 [==============================] - 3s 484us/step - loss: 0.0055 - val_loss: 0.0048
Epoch 306/2000
5336/5336 [==============================] - 3s 484us/step - loss: 0.0055 - val_loss: 0.0048
Epoch 307/2000
5336/5336 [==============================] - 3s 485us/step - loss: 0.0056 - val_loss: 0.0048
Epoch 308/2000
5336/5336 [==============================] - 3s 485us/step - loss: 0.0055 - val_loss: 0.0048
Epoch 309/2000
5336/5336 [==============================] - 3s 487us/step - loss: 0.0055 - val_loss: 0.0048
Epoch 310/2000
5336/5336 [==============================] - 3s 492us/step - loss: 0.0056 - val_loss: 0.0048
Epoch 311/2000
5336/5336 [==============================] - 3s 483us/step - loss: 0.0055 - val_loss: 0.0048
Epoch 312/2000
5336/5336 [==============================] - 3s 485us/step - loss: 0.0056 - val_loss: 0.0048
Epoch 313/2000
5336/5336 [==============================] - 3s 484us/step - loss: 0.0055 - val_loss: 0.0048
Epoch 314/2000
5336/5336 [==

Epoch 381/2000
5336/5336 [==============================] - 3s 484us/step - loss: 0.0055 - val_loss: 0.0048
Epoch 382/2000
5336/5336 [==============================] - 3s 486us/step - loss: 0.0056 - val_loss: 0.0048
Epoch 383/2000
5336/5336 [==============================] - 3s 483us/step - loss: 0.0055 - val_loss: 0.0048
Epoch 384/2000
5336/5336 [==============================] - 3s 496us/step - loss: 0.0055 - val_loss: 0.0048
Epoch 385/2000
5336/5336 [==============================] - 3s 484us/step - loss: 0.0055 - val_loss: 0.0048
Epoch 386/2000
5336/5336 [==============================] - 3s 483us/step - loss: 0.0056 - val_loss: 0.0048
Epoch 387/2000
5336/5336 [==============================] - 3s 481us/step - loss: 0.0056 - val_loss: 0.0048
Epoch 388/2000
5336/5336 [==============================] - 3s 484us/step - loss: 0.0055 - val_loss: 0.0048
Epoch 389/2000
5336/5336 [==============================] - 3s 493us/step - loss: 0.0055 - val_loss: 0.0048
Epoch 390/2000
5336/5336 [==

...

5336/5336 [==============================] - 3s 486us/step - loss: 0.0055 - val_loss: 0.0047
Epoch 1290/2000
5336/5336 [==============================] - 3s 494us/step - loss: 0.0054 - val_loss: 0.0047
Epoch 1291/2000
5336/5336 [==============================] - 3s 485us/step - loss: 0.0055 - val_loss: 0.0047
Epoch 1292/2000
5336/5336 [==============================] - 3s 484us/step - loss: 0.0055 - val_loss: 0.0047
Epoch 1293/2000
5336/5336 [==============================] - 3s 484us/step - loss: 0.0054 - val_loss: 0.0047
Epoch 1294/2000
5336/5336 [==============================] - 3s 496us/step - loss: 0.0055 - val_loss: 0.0047
Epoch 1295/2000
5336/5336 [==============================] - 3s 485us/step - loss: 0.0055 - val_loss: 0.0047
Epoch 1296/2000
5336/5336 [==============================] - 3s 489us/step - loss: 0.0055 - val_loss: 0.0047
Epoch 1297/2000
5336/5336 [==============================] - 3s 486us/step - loss: 0.0054 - val_loss: 0.0047
Epoch 1298/2000
5336/5336 [========

5336/5336 [==============================] - 3s 483us/step - loss: 0.0055 - val_loss: 0.0047
Epoch 1440/2000
5336/5336 [==============================] - 3s 485us/step - loss: 0.0054 - val_loss: 0.0047
Epoch 1441/2000
5336/5336 [==============================] - 3s 491us/step - loss: 0.0055 - val_loss: 0.0047
Epoch 1442/2000
5336/5336 [==============================] - 3s 482us/step - loss: 0.0054 - val_loss: 0.0047
Epoch 1443/2000
5336/5336 [==============================] - 3s 489us/step - loss: 0.0055 - val_loss: 0.0047
Epoch 1444/2000
5336/5336 [==============================] - 3s 488us/step - loss: 0.0054 - val_loss: 0.0047
Epoch 1445/2000
5336/5336 [==============================] - 3s 491us/step - loss: 0.0054 - val_loss: 0.0047
Epoch 1446/2000
5336/5336 [==============================] - 3s 488us/step - loss: 0.0054 - val_loss: 0.0047
Epoch 1447/2000
5336/5336 [==============================] - 3s 485us/step - loss: 0.0055 - val_loss: 0.0047
Epoch 1448/2000
5336/5336 [========

5336/5336 [==============================] - 3s 490us/step - loss: 0.0054 - val_loss: 0.0047
Epoch 1590/2000
5336/5336 [==============================] - 3s 486us/step - loss: 0.0055 - val_loss: 0.0047
Epoch 1591/2000
5336/5336 [==============================] - 3s 481us/step - loss: 0.0054 - val_loss: 0.0047
Epoch 1592/2000
5336/5336 [==============================] - 3s 490us/step - loss: 0.0054 - val_loss: 0.0047
Epoch 1593/2000
5336/5336 [==============================] - 3s 486us/step - loss: 0.0055 - val_loss: 0.0047
Epoch 1594/2000
5336/5336 [==============================] - 3s 488us/step - loss: 0.0054 - val_loss: 0.0047
Epoch 1595/2000
5336/5336 [==============================] - 3s 494us/step - loss: 0.0055 - val_loss: 0.0047
Epoch 1596/2000
5336/5336 [==============================] - 3s 493us/step - loss: 0.0055 - val_loss: 0.0047
Epoch 1597/2000
5336/5336 [==============================] - 3s 486us/step - loss: 0.0055 - val_loss: 0.0047
Epoch 1598/2000
5336/5336 [========

5336/5336 [==============================] - 3s 487us/step - loss: 0.0055 - val_loss: 0.0047
Epoch 1740/2000
5336/5336 [==============================] - 3s 487us/step - loss: 0.0054 - val_loss: 0.0047
Epoch 1741/2000
5336/5336 [==============================] - 3s 494us/step - loss: 0.0055 - val_loss: 0.0047
Epoch 1742/2000
5336/5336 [==============================] - 3s 495us/step - loss: 0.0055 - val_loss: 0.0047
Epoch 1743/2000
5336/5336 [==============================] - 3s 489us/step - loss: 0.0054 - val_loss: 0.0047
Epoch 1744/2000
5336/5336 [==============================] - 3s 490us/step - loss: 0.0055 - val_loss: 0.0047
Epoch 1745/2000
5336/5336 [==============================] - 3s 485us/step - loss: 0.0055 - val_loss: 0.0047
Epoch 1746/2000
5336/5336 [==============================] - 3s 499us/step - loss: 0.0055 - val_loss: 0.0047
Epoch 1747/2000
5336/5336 [==============================] - 3s 487us/step - loss: 0.0055 - val_loss: 0.0047
Epoch 1748/2000
5336/5336 [========

5336/5336 [==============================] - 3s 483us/step - loss: 0.0055 - val_loss: 0.0047
Epoch 1890/2000
5336/5336 [==============================] - 3s 486us/step - loss: 0.0054 - val_loss: 0.0047
Epoch 1891/2000
5336/5336 [==============================] - 3s 484us/step - loss: 0.0055 - val_loss: 0.0047
Epoch 1892/2000
5336/5336 [==============================] - 3s 487us/step - loss: 0.0054 - val_loss: 0.0047
Epoch 1893/2000
5336/5336 [==============================] - 3s 497us/step - loss: 0.0054 - val_loss: 0.0047
Epoch 1894/2000
5336/5336 [==============================] - 3s 489us/step - loss: 0.0055 - val_loss: 0.0047
Epoch 1895/2000
5336/5336 [==============================] - 3s 489us/step - loss: 0.0054 - val_loss: 0.0047
Epoch 1896/2000
5336/5336 [==============================] - 3s 488us/step - loss: 0.0055 - val_loss: 0.0047
Epoch 1897/2000
5336/5336 [==============================] - 3s 501us/step - loss: 0.0055 - val_loss: 0.0047
Epoch 1898/2000
5336/5336 [========

In [318]:
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X_s, y_s, test_size= .2)

In [559]:
# New datasets
model = Sequential()
model.add(Dense(64, input_shape=(230,), activation='relu'))
model.add(Dropout(.25))
model.add(Dense(64, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(128, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(256, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(2000, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(1, activation='relu'))
# best lr .02
sgd = keras.optimizers.SGD(lr=0.01)
model.compile(optimizer=sgd,
              loss='mean_squared_error')

In [560]:
model.fit(x=X_train_m, y=y_train_m, 
          batch_size=2500, 
          epochs=2000, 
          verbose=1, 
          validation_data=(X_test_m, y_test_m),
          callbacks=None)



Train on 2186 samples, validate on 547 samples
Epoch 1/2000
2186/2186 [==============================] - 4s 2ms/step - loss: 1.1081 - val_loss: 1.0686
Epoch 2/2000
2186/2186 [==============================] - 1s 492us/step - loss: 1.0675 - val_loss: 0.9908
Epoch 3/2000
2186/2186 [==============================] - 1s 487us/step - loss: 0.9906 - val_loss: 0.9191
Epoch 4/2000
2186/2186 [==============================] - 1s 490us/step - loss: 0.9190 - val_loss: 0.8509
Epoch 5/2000
2186/2186 [==============================] - 1s 488us/step - loss: 0.8512 - val_loss: 0.7859
Epoch 6/2000
2186/2186 [==============================] - 1s 487us/step - loss: 0.7870 - val_loss: 0.7244
Epoch 7/2000
2186/2186 [==============================] - 1s 489us/step - loss: 0.7261 - val_loss: 0.6666
Epoch 8/2000
2186/2186 [==============================] - 1s 488us/step - loss: 0.6690 - val_loss: 0.6124
Epoch 9/2000
2186/2186 [==============================] - 1s 487us/step - loss: 0.6154 - val_loss: 0.5618
E

Epoch 78/2000
2186/2186 [==============================] - 1s 489us/step - loss: 0.0044 - val_loss: 0.0037
Epoch 79/2000
2186/2186 [==============================] - 1s 488us/step - loss: 0.0043 - val_loss: 0.0037
Epoch 80/2000
2186/2186 [==============================] - 1s 489us/step - loss: 0.0042 - val_loss: 0.0037
Epoch 81/2000
2186/2186 [==============================] - 1s 507us/step - loss: 0.0043 - val_loss: 0.0037
Epoch 82/2000
2186/2186 [==============================] - 1s 488us/step - loss: 0.0043 - val_loss: 0.0037
Epoch 83/2000
2186/2186 [==============================] - 1s 488us/step - loss: 0.0043 - val_loss: 0.0037
Epoch 84/2000
2186/2186 [==============================] - 1s 486us/step - loss: 0.0043 - val_loss: 0.0037
Epoch 85/2000
2186/2186 [==============================] - 1s 488us/step - loss: 0.0041 - val_loss: 0.0037
Epoch 86/2000
2186/2186 [==============================] - 1s 487us/step - loss: 0.0042 - val_loss: 0.0037
Epoch 87/2000
2186/2186 [============

2186/2186 [==============================] - 1s 539us/step - loss: 0.0041 - val_loss: 0.0039
Epoch 155/2000
2186/2186 [==============================] - 1s 491us/step - loss: 0.0041 - val_loss: 0.0039
Epoch 156/2000
2186/2186 [==============================] - 1s 487us/step - loss: 0.0042 - val_loss: 0.0039
Epoch 157/2000
2186/2186 [==============================] - 1s 488us/step - loss: 0.0041 - val_loss: 0.0039
Epoch 158/2000
2186/2186 [==============================] - 1s 493us/step - loss: 0.0041 - val_loss: 0.0039
Epoch 159/2000
2186/2186 [==============================] - 1s 514us/step - loss: 0.0041 - val_loss: 0.0039
Epoch 160/2000
2186/2186 [==============================] - 1s 485us/step - loss: 0.0041 - val_loss: 0.0039
Epoch 161/2000
2186/2186 [==============================] - 1s 487us/step - loss: 0.0042 - val_loss: 0.0039
Epoch 162/2000
2186/2186 [==============================] - 1s 486us/step - loss: 0.0041 - val_loss: 0.0039
Epoch 163/2000
2186/2186 [=================

Epoch 230/2000
2186/2186 [==============================] - 1s 496us/step - loss: 0.0041 - val_loss: 0.0039
Epoch 231/2000
2186/2186 [==============================] - 1s 483us/step - loss: 0.0042 - val_loss: 0.0039
Epoch 232/2000
2186/2186 [==============================] - 1s 485us/step - loss: 0.0042 - val_loss: 0.0039
Epoch 233/2000
2186/2186 [==============================] - 1s 486us/step - loss: 0.0041 - val_loss: 0.0039
Epoch 234/2000
2186/2186 [==============================] - 1s 497us/step - loss: 0.0040 - val_loss: 0.0039
Epoch 235/2000
2186/2186 [==============================] - 1s 486us/step - loss: 0.0042 - val_loss: 0.0039
Epoch 236/2000
2186/2186 [==============================] - 1s 490us/step - loss: 0.0040 - val_loss: 0.0039
Epoch 237/2000
2186/2186 [==============================] - 1s 483us/step - loss: 0.0041 - val_loss: 0.0039
Epoch 238/2000
2186/2186 [==============================] - 1s 486us/step - loss: 0.0041 - val_loss: 0.0039
Epoch 239/2000
2186/2186 [==

Epoch 306/2000
2186/2186 [==============================] - 1s 489us/step - loss: 0.0040 - val_loss: 0.0039
Epoch 307/2000
2186/2186 [==============================] - 1s 486us/step - loss: 0.0041 - val_loss: 0.0039
Epoch 308/2000
2186/2186 [==============================] - 1s 486us/step - loss: 0.0042 - val_loss: 0.0039
Epoch 309/2000
2186/2186 [==============================] - 1s 488us/step - loss: 0.0041 - val_loss: 0.0039
Epoch 310/2000
2186/2186 [==============================] - 1s 487us/step - loss: 0.0041 - val_loss: 0.0039
Epoch 311/2000
2186/2186 [==============================] - 1s 490us/step - loss: 0.0042 - val_loss: 0.0039
Epoch 312/2000
2186/2186 [==============================] - 1s 487us/step - loss: 0.0040 - val_loss: 0.0039
Epoch 313/2000
2186/2186 [==============================] - 1s 483us/step - loss: 0.0042 - val_loss: 0.0039
Epoch 314/2000
2186/2186 [==============================] - 1s 511us/step - loss: 0.0041 - val_loss: 0.0039
Epoch 315/2000
2186/2186 [==

...

Epoch 686/2000
2186/2186 [==============================] - 1s 491us/step - loss: 0.0039 - val_loss: 0.0039
Epoch 687/2000
2186/2186 [==============================] - 1s 488us/step - loss: 0.0040 - val_loss: 0.0039
Epoch 688/2000
2186/2186 [==============================] - 1s 486us/step - loss: 0.0040 - val_loss: 0.0039
Epoch 689/2000
2186/2186 [==============================] - 1s 517us/step - loss: 0.0041 - val_loss: 0.0039
Epoch 690/2000
2186/2186 [==============================] - 1s 489us/step - loss: 0.0040 - val_loss: 0.0039
Epoch 691/2000
2186/2186 [==============================] - 1s 486us/step - loss: 0.0040 - val_loss: 0.0039
Epoch 692/2000
2186/2186 [==============================] - 1s 488us/step - loss: 0.0041 - val_loss: 0.0039
Epoch 693/2000
2186/2186 [==============================] - 1s 491us/step - loss: 0.0041 - val_loss: 0.0039
Epoch 694/2000
2186/2186 [==============================] - 1s 497us/step - loss: 0.0042 - val_loss: 0.0039
Epoch 695/2000
2186/2186 [==

Epoch 762/2000
2186/2186 [==============================] - 1s 488us/step - loss: 0.0041 - val_loss: 0.0039
Epoch 763/2000
2186/2186 [==============================] - 1s 513us/step - loss: 0.0041 - val_loss: 0.0039
Epoch 764/2000
2186/2186 [==============================] - 1s 494us/step - loss: 0.0041 - val_loss: 0.0039
Epoch 765/2000
2186/2186 [==============================] - 1s 489us/step - loss: 0.0040 - val_loss: 0.0039
Epoch 766/2000
2186/2186 [==============================] - 1s 488us/step - loss: 0.0041 - val_loss: 0.0039
Epoch 767/2000
2186/2186 [==============================] - 1s 494us/step - loss: 0.0040 - val_loss: 0.0039
Epoch 768/2000
2186/2186 [==============================] - 1s 486us/step - loss: 0.0040 - val_loss: 0.0039
Epoch 769/2000
2186/2186 [==============================] - 1s 490us/step - loss: 0.0040 - val_loss: 0.0039
Epoch 770/2000
2186/2186 [==============================] - 1s 489us/step - loss: 0.0041 - val_loss: 0.0039
Epoch 771/2000
2186/2186 [==

Epoch 838/2000
2186/2186 [==============================] - 1s 486us/step - loss: 0.0040 - val_loss: 0.0039
Epoch 839/2000
2186/2186 [==============================] - 1s 488us/step - loss: 0.0041 - val_loss: 0.0039
Epoch 840/2000
2186/2186 [==============================] - 1s 484us/step - loss: 0.0041 - val_loss: 0.0039
Epoch 841/2000
2186/2186 [==============================] - 1s 485us/step - loss: 0.0039 - val_loss: 0.0039
Epoch 842/2000
2186/2186 [==============================] - 1s 488us/step - loss: 0.0039 - val_loss: 0.0039
Epoch 843/2000
2186/2186 [==============================] - 1s 491us/step - loss: 0.0041 - val_loss: 0.0039
Epoch 844/2000
2186/2186 [==============================] - 1s 488us/step - loss: 0.0040 - val_loss: 0.0039
Epoch 845/2000
2186/2186 [==============================] - 1s 486us/step - loss: 0.0040 - val_loss: 0.0039
Epoch 846/2000
2186/2186 [==============================] - 1s 487us/step - loss: 0.0040 - val_loss: 0.0039
Epoch 847/2000
2186/2186 [==

Epoch 914/2000
2186/2186 [==============================] - 1s 484us/step - loss: 0.0041 - val_loss: 0.0039
Epoch 915/2000
2186/2186 [==============================] - 1s 492us/step - loss: 0.0040 - val_loss: 0.0039
Epoch 916/2000
2186/2186 [==============================] - 1s 493us/step - loss: 0.0041 - val_loss: 0.0039
Epoch 917/2000
2186/2186 [==============================] - 1s 486us/step - loss: 0.0040 - val_loss: 0.0039
Epoch 918/2000
2186/2186 [==============================] - 1s 486us/step - loss: 0.0040 - val_loss: 0.0039
Epoch 919/2000
2186/2186 [==============================] - 1s 490us/step - loss: 0.0040 - val_loss: 0.0039
Epoch 920/2000
2186/2186 [==============================] - 1s 490us/step - loss: 0.0040 - val_loss: 0.0039
Epoch 921/2000
2186/2186 [==============================] - 1s 570us/step - loss: 0.0040 - val_loss: 0.0039
Epoch 922/2000
2186/2186 [==============================] - 1s 620us/step - loss: 0.0041 - val_loss: 0.0039
Epoch 923/2000
2186/2186 [==

KeyboardInterrupt: 

In [ ]:
root = .0032 ** .5

In [225]:
root

0.058309518948453

In [561]:
# new datasets with new features (big swing)
model = Sequential()
model.add(Dense(64, input_shape=(450,), activation='relu'))
model.add(Dropout(.25))
model.add(Dense(64, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(128, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(256, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(2000, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(1, activation='relu'))

sgd = keras.optimizers.SGD(lr=0.01)
model.compile(optimizer=sgd,
              loss='mean_squared_error')

In [562]:
model.fit(x=X_train_sm, y=y_train_sm, 
          batch_size=2500, 
          epochs=2000, 
          verbose=1, 
          validation_data=(X_test_sm, y_test_sm),
          callbacks=None)



Train on 2186 samples, validate on 547 samples
Epoch 1/2000
2186/2186 [==============================] - 3s 2ms/step - loss: 1.1054 - val_loss: 1.0352
Epoch 2/2000
2186/2186 [==============================] - 1s 502us/step - loss: 1.0320 - val_loss: 0.9584
Epoch 3/2000
2186/2186 [==============================] - 1s 519us/step - loss: 0.9561 - val_loss: 0.8839
Epoch 4/2000
2186/2186 [==============================] - 1s 496us/step - loss: 0.8826 - val_loss: 0.8130
Epoch 5/2000
2186/2186 [==============================] - 1s 498us/step - loss: 0.8131 - val_loss: 0.7462
Epoch 6/2000
2186/2186 [==============================] - 1s 506us/step - loss: 0.7475 - val_loss: 0.6837
Epoch 7/2000
2186/2186 [==============================] - 1s 508us/step - loss: 0.6858 - val_loss: 0.6255
Epoch 8/2000
2186/2186 [==============================] - 1s 505us/step - loss: 0.6284 - val_loss: 0.5714
Epoch 9/2000
2186/2186 [==============================] - 1s 500us/step - loss: 0.5750 - val_loss: 0.5213
E

Epoch 78/2000
2186/2186 [==============================] - 1s 493us/step - loss: 0.0043 - val_loss: 0.0035
Epoch 79/2000
2186/2186 [==============================] - 1s 491us/step - loss: 0.0043 - val_loss: 0.0035
Epoch 80/2000
2186/2186 [==============================] - 1s 492us/step - loss: 0.0042 - val_loss: 0.0035
Epoch 81/2000
2186/2186 [==============================] - 1s 489us/step - loss: 0.0042 - val_loss: 0.0035
Epoch 82/2000
2186/2186 [==============================] - 1s 493us/step - loss: 0.0043 - val_loss: 0.0035
Epoch 83/2000
2186/2186 [==============================] - 1s 492us/step - loss: 0.0041 - val_loss: 0.0035
Epoch 84/2000
2186/2186 [==============================] - 1s 493us/step - loss: 0.0043 - val_loss: 0.0036
Epoch 85/2000
2186/2186 [==============================] - 1s 508us/step - loss: 0.0042 - val_loss: 0.0036
Epoch 86/2000
2186/2186 [==============================] - 1s 499us/step - loss: 0.0042 - val_loss: 0.0036
Epoch 87/2000
2186/2186 [============

KeyboardInterrupt: 

In [563]:
# new datasets with new features (big swing) and new layers
model = Sequential()
model.add(Dense(64, input_shape=(450,), activation='relu'))
model.add(Dropout(.25))
model.add(Dense(64, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(128, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(256, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(2000, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(2000, activation='relu'))
model.add(Dense(4000, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(1, activation='linear'))

sgd = keras.optimizers.SGD(lr=0.01)
model.compile(optimizer=sgd,
              loss='mean_squared_error')

In [ ]:
model.fit(x=X_train_sm, y=y_train_sm, 
          batch_size=2500, 
          epochs=2000, 
          verbose=1, 
          validation_data=(X_test_sm, y_test_sm),
          callbacks=None)

Train on 2186 samples, validate on 547 samples
Epoch 1/2000
2186/2186 [==============================] - 9s 4ms/step - loss: 1.1104 - val_loss: 1.0365
Epoch 2/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 1.0305 - val_loss: 0.9589
Epoch 3/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.9543 - val_loss: 0.8822
Epoch 4/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.8791 - val_loss: 0.8091
Epoch 5/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.8070 - val_loss: 0.7405
Epoch 6/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.7393 - val_loss: 0.6766
Epoch 7/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.6760 - val_loss: 0.6172
Epoch 8/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.6172 - val_loss: 0.5622
Epoch 9/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.5626 - val_loss: 0.5114
Epoch 10/2000
218

Epoch 79/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0038 - val_loss: 0.0034
Epoch 80/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0038 - val_loss: 0.0034
Epoch 81/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0039 - val_loss: 0.0034
Epoch 82/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0038 - val_loss: 0.0034
Epoch 83/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0038 - val_loss: 0.0034
Epoch 84/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0038 - val_loss: 0.0034
Epoch 85/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0039 - val_loss: 0.0034
Epoch 86/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0037 - val_loss: 0.0034
Epoch 87/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0038 - val_loss: 0.0034
Epoch 88/2000
2186/2186 [==============================

Epoch 157/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0037 - val_loss: 0.0035
Epoch 158/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 159/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 160/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 161/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 162/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 163/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 164/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 165/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0039 - val_loss: 0.0035
Epoch 166/2000
2186/2186 [====================

Epoch 235/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 236/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 237/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 238/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 239/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 240/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 241/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 242/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 243/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 244/2000
2186/2186 [====================

Epoch 313/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 314/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 315/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 316/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 317/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 318/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0037 - val_loss: 0.0035
Epoch 319/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 320/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 321/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 322/2000
2186/2186 [====================

Epoch 391/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 392/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 393/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0037 - val_loss: 0.0035
Epoch 394/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 395/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 396/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 397/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0037 - val_loss: 0.0035
Epoch 398/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 399/2000
2186/2186 [==============================] - 6s 3ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 400/2000
2186/2186 [====================

In [ ]:
# new datasets with new features (big swing) and new layers
model = Sequential()
model.add(Dense(64, input_shape=(450,), activation='relu'))
model.add(Dropout(.25))
model.add(Dense(64, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(128, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(256, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(2000, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(2000, activation='relu'))
model.add(Dense(4000, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(1, activation='linear'))

sgd = keras.optimizers.SGD(lr=0.0001)
model.compile(optimizer=sgd,
              loss='mean_squared_error')

In [ ]:
model.fit(x=X_train_sm, y=y_train_sm, 
          batch_size=2500, 
          epochs=2000, 
          verbose=1, 
          validation_data=(X_test_sm, y_test_sm),
          callbacks=None)